Aqui daremos toda introdução ao problema e explicaremos o que os dados falam, qual será o problema a ser resolvido, quais coluna iremos utilizar, tudo sobre contexto e introdução ao assunto. Virá antes da parte de ETL.

Para um portfólio de alto nível (especialmente mirando bancos), a melhor resposta é uma Abordagem Híbrida, mas com papéis bem definidos para cada lugar.

Não jogue tudo no README (fica poluído) e nem esconda a lógica de negócios apenas no código.

Aqui está a estrutura "Padrão Ouro" que sugiro para você:

1. O Papel do README.md (O "Sumário Executivo")
O README é para o Recrutador/Gerente. Ele quer saber o Contexto de Negócio, não o detalhe de qual coluna você escolheu.

O que escrever aqui:

Contexto: "O Home Credit Group quer oferecer empréstimos para pessoas com pouco ou nenhum histórico bancário..."

O Problema: "Muitas pessoas são rejeitadas injustamente. Precisamos usar dados alternativos para avaliar esse risco."

A Estratégia (Resumo): "Neste projeto, focamos na tabela principal (application_train) analisando variáveis socioeconômicas (renda, educação) e externas."

2. O Papel do Notebook 01_Data_Understanding.ipynb (A "Mão na Massa")
Aqui é onde você brilha tecnicamente. Em vez de começar direto limpando (ETL), crie um notebook inicial focado no Entendimento dos Dados (Seguindo a metodologia CRISP-DM, muito usada em bancos).

É nesse notebook que você vai fazer o estudo das variáveis que comentou.

Sugestão de Estrutura para este Notebook:

Célula 1 [Markdown]: Introdução e Hipóteses
Escreva quais perguntas você quer responder.

Ex: "Hipótese 1: Pessoas com maior renda têm menor risco de inadimplência?"

Ex: "Hipótese 2: O tipo de contrato de trabalho influencia no pagamento?"

Célula 2 [Code]: Carregando o Dicionário de Dados
O dataset do Home Credit vem com um arquivo chamado HomeCredit_columns_description.csv. Isso é ouro. Em vez de adivinhar o que é a coluna AMT_ANNUITY, você carrega esse arquivo e mostra a definição.

Python
import pandas as pd
# Carregando as descrições para entender as variáveis
description = pd.read_csv('../data/raw/HomeCredit_columns_description.csv', encoding='latin1')

# Criando uma funçãozinha para consultar o que significa uma coluna
def explicar_coluna(nome_coluna):
    row = description[description['Row'] == nome_coluna]
    print(f"Definição: {row['Description'].values[0]}")

explicar_coluna('AMT_INCOME_TOTAL')
Célula 3 [Markdown/Code]: Seleção de Variáveis (Feature Selection Manual)
Aqui você escreve: "Dentre as 122 colunas, selecionaremos as seguintes para a análise inicial, divididas em grupos:"

Dados Financeiros: AMT_INCOME_TOTAL, AMT_CREDIT, AMT_ANNUITY...

Dados Demográficos: CODE_GENDER, CNT_CHILDREN, NAME_EDUCATION_TYPE...

Score Externo: EXT_SOURCE_1, EXT_SOURCE_2...

Resumo da Estratégia
No README: Coloque o "Porquê" do projeto e o objetivo final.

No Notebook 01 (Data Understanding): Coloque o estudo detalhado, a leitura do dicionário de dados e a lista das variáveis escolhidas.

Isso mostra que você não saiu codando loucamente. Mostra que você parou, leu a documentação, formulou hipóteses e selecionou as ferramentas (variáveis) certas. Isso é Maturidade Analítica.

Pode criar seu primeiro notebook com o nome 01_data_understanding_and_scope.ipynb. O que acha?

# Introdução e Hipótese

Inicialmente utilizaremos os dados de `HomeCredit_columns_description.csv` para entender mais sobre os nosso dados e selecionar os que podem ser mais relevantes. Ao final escolheremos os dados utilizados e criaremos algumas hipóteses que serão respondidas no final dos Notebooks

Inicialmente, utilizaremos apenas a tabela `application_train.csv` pois ela é uma tabela rica que permitirá obter insights valiosos dos dados

## Importação de Bibliotecas

In [7]:
import pandas as pd

In [8]:
description_df = pd.read_csv('../data/HomeCredit_columns_description.csv', sep=',')
description_df.head()

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN


In [ ]:
# Creating a function to filter descriptions for specific tables
def filter_description(df):
    
    selected_table = 'application_{train|test}.csv'

    return (df[df['Table'] == selected_table]
            .drop(columns=['Table', 'Unnamed: 0'])
            )

In [37]:
description_list = filter_description(description_df)
description_list

,Description,Special
Row,,
SK_ID_CURR,ID of loan in our sample,NaN
TARGET,Target variable (1 - client with payment diffi...,NaN
NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
CODE_GENDER,Gender of the client,NaN
FLAG_OWN_CAR,Flag if the client owns a car,NaN
...,...,...
AMT_REQ_CREDIT_BUREAU_DAY,Number of enquiries to Credit Bureau about the...,NaN
AMT_REQ_CREDIT_BUREAU_WEEK,Number of enquiries to Credit Bureau about the...,NaN
AMT_REQ_CREDIT_BUREAU_MON,Number of enquiries to Credit Bureau about the...,NaN


Aqui, visando aplicação dos conheicmentos e pensando em estratégias de negócio escolheremos as colunas pensando nos 5 C's: (Character, Capacity, Capital, Collateral, Conditions)

1. **Variáveis Essenciais**
    - `TARGET`: A variável alvo (0 = Todos os outros casos, 1 = Dificuldades com o pagamento)
    - `SK_ID_CURR`: Identificado único do empréstimo
    - `NAME_CONTRACT_TYPE`: Tipo de empréstimo

2. **Character**: Aqui analisaremos a saúde financeira do cliente
    - `AMT_INCOME_TOTAL`: Renda total
    - `AMT_CREDIT`: Valor do empréstimo pedido
    - `AMT_ANNUITY`: Valor da parcela anual
    - `AMT_GOODS_PRICE`: Preço do Bem (Caso seja um financiamento de bens específicos)

3. **Perfil do Cliente**
    - `CODE_GENDER`: Gênero do cliente
    - `DAYS_BIRTH`: Idade em dias
    - `NAME_EDUCATION_TYPE`: Maior nível de escolaridade (É uma variável "Ordinal" (Ensino Médio < Superior < Pós). Você pode explorar Label Encoding aqui.)
    - `NAME_FAMILY_STATUS`: Status Civil
    - `CNT_CHILDREN`: Quantidade de filhos
    - `REGION_RATING_CLIENT`: Região de moradia (1, 2 e 3)

4. **Estabilidade e Trabalho**
    - `DAYS_EMPLOYED`: Dias empregado (Pegadinha Famosa (Bônus para seu Portfólio): Essa coluna tem um valor "mágico" (365243) para desempregados/aposentados que distorce a média. Descobrir e tratar isso na sua EDA mostra nível Senior.)
    - `NAME_INCOME_TYPE`: Tipo de renda (Empresário, assalariado, pensionista, etc)
    - `OCCUPATION_TYPE`: Profissão

5. **Ativos e Score Externo**
    - `FLAG_OWN_CAR`: Flag para se o cliente possui um carro
    - `FLAG_OWN_REALTY`: Flag para se o cliente possui uma casa
    - `OWN_CAR_AGE`: Idade do carro
    - `EXT_SOURCE_1`: Score normalizado de outra agência de crédito
    - `EXT_SOURCE_2`: Score normalizado de outra agência de crédito
    - `EXT_SOURCE_3`: Score normalizado de outra agência de crédito


Feature-engineering:
- Debt-to-Income Ratio = AMT_ANNUITY / AMT_INCOME_TOTAL (Quanto da renda está comprometida?)

- Credit-Term = AMT_CREDIT / AMT_ANNUITY (Quanto tempo vai demorar para pagar?